<a href="https://colab.research.google.com/github/dimitarpg13/agentic_architectures_and_design_patterns/blob/main/notebooks/model_evaluation/mlflow_flesch_kincaid_metric_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLflow Flesch-Kincaid Readability Metrics Demonstration

This notebook provides a comprehensive guide to using Flesch-Kincaid readability metrics with MLflow for evaluating text complexity and readability in NLP applications.

## What are Flesch-Kincaid Metrics?

The Flesch-Kincaid metrics are two related tests that assess the readability of English text:

### 1. Flesch Reading Ease (FRE)
- **Range**: 0-100 (higher = easier to read)
- **Formula**: 206.835 - 1.015 × (total words/total sentences) - 84.6 × (total syllables/total words)
- **Interpretation**:
  - 90-100: Very Easy (5th grade)
  - 80-89: Easy (6th grade)
  - 70-79: Fairly Easy (7th grade)
  - 60-69: Standard (8th-9th grade)
  - 50-59: Fairly Difficult (10th-12th grade)
  - 30-49: Difficult (College)
  - 0-29: Very Difficult (College graduate)

### 2. Flesch-Kincaid Grade Level (FKGL)
- **Range**: Typically 0-18+ (U.S. grade level)
- **Formula**: 0.39 × (total words/total sentences) + 11.8 × (total syllables/total words) - 15.59
- **Interpretation**: Direct mapping to U.S. education grade levels

## 1. Installation and Setup

In [ ]:
# Install required packages
!pip install -q mlflow textstat pyphen nltk pandas numpy matplotlib seaborn plotly scipy

In [ ]:
import mlflow
import mlflow.metrics
from mlflow.metrics import make_metric, MetricValue
import pandas as pd
import numpy as np
import json
from typing import List, Dict, Tuple, Optional
import warnings
import re
warnings.filterwarnings('ignore')

# Text analysis libraries
import textstat
import pyphen
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import cmudict

# Download required NLTK data
nltk.download('punkt', quiet=True)
nltk.download('cmudict', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Statistics
from scipy import stats

sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

print(f"MLflow version: {mlflow.__version__}")
print(f"Textstat version: {textstat.__version__}")

## 2. Creating Custom Flesch-Kincaid Metrics for MLflow

In [ ]:
class FleschKincaidMetrics:
    """
    Comprehensive Flesch-Kincaid readability metrics for MLflow.
    """

    def __init__(self):
        # Initialize CMU Pronouncing Dictionary for syllable counting
        self.cmu_dict = cmudict.dict()
        self.pyphen_dic = pyphen.Pyphen(lang='en')

    def count_syllables(self, word: str) -> int:
        """
        Count syllables in a word using multiple methods for accuracy.
        """
        word = word.lower().strip()

        # Try CMU dictionary first (most accurate)
        if word in self.cmu_dict:
            return len([ph for ph in self.cmu_dict[word][0] if ph[-1].isdigit()])

        # Fallback to pyphen
        hyphenated = self.pyphen_dic.inserted(word)
        return len(hyphenated.split('-'))

    def calculate_text_statistics(self, text: str) -> Dict:
        """
        Calculate comprehensive text statistics needed for readability metrics.
        """
        # Clean text
        text = re.sub(r'\s+', ' ', text.strip())

        # Tokenize
        sentences = sent_tokenize(text)
        words = word_tokenize(text)

        # Filter words (remove punctuation)
        words = [w for w in words if w.isalnum()]

        # Count syllables
        total_syllables = sum(self.count_syllables(word) for word in words)

        # Calculate statistics
        num_sentences = len(sentences)
        num_words = len(words)

        # Avoid division by zero
        if num_sentences == 0 or num_words == 0:
            return {
                'num_sentences': 0,
                'num_words': 0,
                'num_syllables': 0,
                'avg_words_per_sentence': 0,
                'avg_syllables_per_word': 0
            }

        return {
            'num_sentences': num_sentences,
            'num_words': num_words,
            'num_syllables': total_syllables,
            'avg_words_per_sentence': num_words / num_sentences,
            'avg_syllables_per_word': total_syllables / num_words
        }

    def flesch_reading_ease(self, text: str) -> float:
        """
        Calculate Flesch Reading Ease score.
        Score range: 0-100 (higher = easier to read)
        """
        stats = self.calculate_text_statistics(text)

        if stats['num_words'] == 0 or stats['num_sentences'] == 0:
            return 0.0

        score = 206.835 - (1.015 * stats['avg_words_per_sentence']) - \
                (84.6 * stats['avg_syllables_per_word'])

        # Clamp to 0-100 range
        return max(0, min(100, score))

    def flesch_kincaid_grade(self, text: str) -> float:
        """
        Calculate Flesch-Kincaid Grade Level.
        Returns U.S. school grade level.
        """
        stats = self.calculate_text_statistics(text)

        if stats['num_words'] == 0 or stats['num_sentences'] == 0:
            return 0.0

        grade = (0.39 * stats['avg_words_per_sentence']) + \
                (11.8 * stats['avg_syllables_per_word']) - 15.59

        # Typically clamped to minimum of -3.4 (kindergarten) and maximum of 18+
        return max(-3.4, min(18, grade))

    def create_reading_ease_metric(self):
        """
        Create MLflow metric for Flesch Reading Ease.
        """
        def reading_ease_metric(predictions, targets=None):
            if not isinstance(predictions, list):
                predictions = [predictions]

            scores = [self.flesch_reading_ease(text) for text in predictions]

            return MetricValue(
                aggregate_results={
                    "flesch_reading_ease_mean": np.mean(scores),
                    "flesch_reading_ease_std": np.std(scores),
                    "flesch_reading_ease_min": np.min(scores),
                    "flesch_reading_ease_max": np.max(scores)
                },
                scores=scores
            )

        return make_metric(
            eval_fn=reading_ease_metric,
            greater_is_better=True,  # Higher scores = easier to read
            name="flesch_reading_ease"
        )

    def create_grade_level_metric(self):
        """
        Create MLflow metric for Flesch-Kincaid Grade Level.
        """
        def grade_level_metric(predictions, targets=None):
            if not isinstance(predictions, list):
                predictions = [predictions]

            scores = [self.flesch_kincaid_grade(text) for text in predictions]

            return MetricValue(
                aggregate_results={
                    "flesch_kincaid_grade_mean": np.mean(scores),
                    "flesch_kincaid_grade_std": np.std(scores),
                    "flesch_kincaid_grade_min": np.min(scores),
                    "flesch_kincaid_grade_max": np.max(scores)
                },
                scores=scores
            )

        return make_metric(
            eval_fn=grade_level_metric,
            greater_is_better=False,  # Lower grade = more accessible
            name="flesch_kincaid_grade"
        )

    def create_comprehensive_readability_metric(self):
        """
        Create a comprehensive readability metric including multiple measures.
        """
        def comprehensive_metric(predictions, targets=None):
            if not isinstance(predictions, list):
                predictions = [predictions]

            all_metrics = []
            for text in predictions:
                metrics = {
                    'flesch_reading_ease': self.flesch_reading_ease(text),
                    'flesch_kincaid_grade': self.flesch_kincaid_grade(text),
                    'gunning_fog': textstat.gunning_fog(text),
                    'smog_index': textstat.smog_index(text),
                    'coleman_liau_index': textstat.coleman_liau_index(text),
                    'automated_readability_index': textstat.automated_readability_index(text),
                    'dale_chall_readability': textstat.dale_chall_readability_score(text),
                    'linsear_write_formula': textstat.linsear_write_formula(text)
                }
                all_metrics.append(metrics)

            # Calculate aggregates
            aggregate_results = {}
            for key in all_metrics[0].keys():
                values = [m[key] for m in all_metrics]
                aggregate_results[f"{key}_mean"] = np.mean(values)
                aggregate_results[f"{key}_std"] = np.std(values)

            # Add consensus grade level
            aggregate_results['consensus_grade'] = np.mean([
                m['flesch_kincaid_grade'] for m in all_metrics
            ])

            return MetricValue(
                aggregate_results=aggregate_results,
                scores=[m['flesch_reading_ease'] for m in all_metrics]
            )

        return make_metric(
            eval_fn=comprehensive_metric,
            greater_is_better=True,
            name="comprehensive_readability"
        )

# Initialize metrics
fk_metrics = FleschKincaidMetrics()
reading_ease_metric = fk_metrics.create_reading_ease_metric()
grade_level_metric = fk_metrics.create_grade_level_metric()
comprehensive_metric = fk_metrics.create_comprehensive_readability_metric()

print("✅ Custom Flesch-Kincaid metrics created successfully!")

## 3. Preparing Sample Text Data with Various Complexity Levels

In [ ]:
# Create sample texts with different readability levels
sample_texts = [
    {
        "category": "Children's Book",
        "target_grade": "2-3",
        "text": """The cat sat on the mat. It was a sunny day. The cat was happy.
        Birds sang in the trees. The cat watched them fly. It was fun to watch birds.
        The mat was soft and warm. The cat took a nap."""
    },
    {
        "category": "Young Adult Novel",
        "target_grade": "6-8",
        "text": """Sarah walked through the forest, listening to the sounds around her.
        The leaves rustled gently in the breeze, and birds called from the treetops.
        She had been hiking for two hours, following the winding trail that led to the
        waterfall. Her backpack contained everything she needed for the day's adventure."""
    },
    {
        "category": "News Article",
        "target_grade": "8-10",
        "text": """The city council voted yesterday to approve a new budget that allocates
        significant funding for infrastructure improvements. The decision, which passed
        with a narrow majority, will result in road repairs and upgraded public
        transportation systems throughout the metropolitan area. Critics argue that the
        spending could have been better directed toward education initiatives."""
    },
    {
        "category": "Academic Paper",
        "target_grade": "College",
        "text": """The implementation of sophisticated machine learning algorithms has
        revolutionized the methodological approaches utilized in contemporary computational
        linguistics. Through the systematic application of transformer-based architectures,
        researchers have demonstrated unprecedented capabilities in natural language
        understanding and generation, thereby facilitating advancements in automated
        translation systems and sentiment analysis frameworks."""
    },
    {
        "category": "Legal Document",
        "target_grade": "Graduate",
        "text": """Notwithstanding any provisions contained herein to the contrary, the
        aforementioned contractual obligations shall remain enforceable pursuant to the
        applicable jurisdictional statutes, provided that such enforcement does not
        contravene the fundamental principles of equity and fairness as established by
        precedential determinations in relevant appellate court decisions."""
    },
    {
        "category": "Technical Manual",
        "target_grade": "10-12",
        "text": """To configure the network settings, navigate to the system preferences
        panel and select the network configuration option. Enter the IP address, subnet
        mask, and gateway information provided by your network administrator. Ensure that
        the DNS server addresses are correctly specified to enable proper domain name
        resolution. Apply the settings and restart the network service if necessary."""
    }
]

# Convert to DataFrame and calculate initial metrics
df_texts = pd.DataFrame(sample_texts)

# Calculate readability scores for each text
df_texts['reading_ease'] = df_texts['text'].apply(fk_metrics.flesch_reading_ease)
df_texts['grade_level'] = df_texts['text'].apply(fk_metrics.flesch_kincaid_grade)
df_texts['word_count'] = df_texts['text'].apply(lambda x: len(word_tokenize(x)))

print("📊 Sample Text Analysis:")
print("="*80)
for _, row in df_texts.iterrows():
    print(f"\nCategory: {row['category']}")
    print(f"Target Grade: {row['target_grade']}")
    print(f"Flesch Reading Ease: {row['reading_ease']:.1f}")
    print(f"Flesch-Kincaid Grade: {row['grade_level']:.1f}")
    print(f"Word Count: {row['word_count']}")

## 4. Evaluating Text Generation Models with MLflow

In [ ]:
# Set up MLflow experiment
mlflow.set_experiment("flesch-kincaid-metrics-demo")

def evaluate_text_readability(model_name: str,
                             texts: List[str],
                             target_audience: str = "general",
                             desired_grade_range: Tuple[float, float] = None):
    """
    Evaluate text readability using Flesch-Kincaid metrics and log to MLflow.
    """
    with mlflow.start_run(run_name=model_name):
        # Log parameters
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("num_texts", len(texts))
        mlflow.log_param("target_audience", target_audience)
        if desired_grade_range:
            mlflow.log_param("desired_grade_min", desired_grade_range[0])
            mlflow.log_param("desired_grade_max", desired_grade_range[1])

        # Create evaluation dataframe
        eval_df = pd.DataFrame({"predictions": texts})

        # Evaluate with all readability metrics
        metrics = [reading_ease_metric, grade_level_metric, comprehensive_metric]

        results = mlflow.evaluate(
            data=eval_df,
            predictions="predictions",
            extra_metrics=metrics,
            evaluators="default"
        )

        # Calculate additional detailed metrics
        detailed_scores = calculate_detailed_readability(texts)

        # Log detailed metrics
        for key, value in detailed_scores.items():
            mlflow.log_metric(key, value)

        # Check if texts meet target grade range
        if desired_grade_range:
            grades = [fk_metrics.flesch_kincaid_grade(text) for text in texts]
            in_range = sum(desired_grade_range[0] <= g <= desired_grade_range[1]
                          for g in grades)
            mlflow.log_metric("texts_in_target_range", in_range)
            mlflow.log_metric("percentage_in_range", (in_range / len(texts)) * 100)

        return results.metrics, detailed_scores

def calculate_detailed_readability(texts: List[str]) -> Dict:
    """
    Calculate comprehensive readability statistics.
    """
    scores = {}

    # Calculate various readability metrics
    fre_scores = [fk_metrics.flesch_reading_ease(text) for text in texts]
    fkg_scores = [fk_metrics.flesch_kincaid_grade(text) for text in texts]

    # Basic statistics
    scores['avg_reading_ease'] = np.mean(fre_scores)
    scores['std_reading_ease'] = np.std(fre_scores)
    scores['avg_grade_level'] = np.mean(fkg_scores)
    scores['std_grade_level'] = np.std(fkg_scores)

    # Readability distribution
    scores['very_easy_texts'] = sum(1 for s in fre_scores if s >= 90)
    scores['easy_texts'] = sum(1 for s in fre_scores if 80 <= s < 90)
    scores['standard_texts'] = sum(1 for s in fre_scores if 60 <= s < 80)
    scores['difficult_texts'] = sum(1 for s in fre_scores if 30 <= s < 60)
    scores['very_difficult_texts'] = sum(1 for s in fre_scores if s < 30)

    # Grade level distribution
    scores['elementary_level'] = sum(1 for g in fkg_scores if g <= 6)
    scores['middle_school_level'] = sum(1 for g in fkg_scores if 6 < g <= 9)
    scores['high_school_level'] = sum(1 for g in fkg_scores if 9 < g <= 12)
    scores['college_level'] = sum(1 for g in fkg_scores if g > 12)

    return scores

# Simulate different text generation models
model_outputs = {
    "Simple_Text_Generator": {
        "texts": [
            "The dog runs fast. He likes to play. The ball is red.",
            "I go to school. My teacher is nice. We learn new things every day.",
            "The sun is bright today. Birds fly in the sky. Trees are green."
        ],
        "target_audience": "children",
        "grade_range": (1, 4)
    },
    "Standard_Text_Generator": {
        "texts": [
            "The research demonstrates significant improvements in model performance.",
            "Our analysis reveals interesting patterns in consumer behavior.",
            "The implementation of new policies has resulted in measurable outcomes."
        ],
        "target_audience": "general",
        "grade_range": (8, 12)
    },
    "Academic_Text_Generator": {
        "texts": [
            "The epistemological implications of quantum mechanics necessitate a fundamental reconceptualization of deterministic paradigms.",
            "Methodological innovations in computational linguistics facilitate unprecedented analytical capabilities.",
            "The heterogeneous nature of socioeconomic variables complicates predictive modeling frameworks."
        ],
        "target_audience": "academic",
        "grade_range": (14, 18)
    }
}

# Evaluate each model
evaluation_results = {}
for model_name, config in model_outputs.items():
    metrics, detailed = evaluate_text_readability(
        model_name,
        config["texts"],
        config["target_audience"],
        config["grade_range"]
    )
    evaluation_results[model_name] = {"metrics": metrics, "detailed": detailed}

print("✅ Evaluation completed for all text generators!")

## 5. Comprehensive Results Analysis

In [ ]:
# Create comparison table
comparison_data = []

for model_name, results in evaluation_results.items():
    config = model_outputs[model_name]
    row = {
        "Model": model_name,
        "Target Audience": config["target_audience"],
        "Target Grade": f"{config['grade_range'][0]}-{config['grade_range'][1]}",
        "Avg Reading Ease": results["detailed"]["avg_reading_ease"],
        "Avg Grade Level": results["detailed"]["avg_grade_level"],
        "Std Grade Level": results["detailed"]["std_grade_level"]
    }
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)

print("\n📊 Model Comparison Results:")
print("="*90)
print(comparison_df.round(2).to_string(index=False))

# Analyze readability distribution
print("\n📈 Readability Distribution Analysis:")
print("="*90)

for model_name, results in evaluation_results.items():
    detailed = results["detailed"]
    print(f"\n{model_name}:")

    # Reading ease categories
    total_texts = sum([
        detailed.get('very_easy_texts', 0),
        detailed.get('easy_texts', 0),
        detailed.get('standard_texts', 0),
        detailed.get('difficult_texts', 0),
        detailed.get('very_difficult_texts', 0)
    ])

    if total_texts > 0:
        print(f"  Very Easy: {detailed.get('very_easy_texts', 0)}/{total_texts}")
        print(f"  Easy: {detailed.get('easy_texts', 0)}/{total_texts}")
        print(f"  Standard: {detailed.get('standard_texts', 0)}/{total_texts}")
        print(f"  Difficult: {detailed.get('difficult_texts', 0)}/{total_texts}")
        print(f"  Very Difficult: {detailed.get('very_difficult_texts', 0)}/{total_texts}")

## 6. Advanced Visualizations

In [ ]:
# Create comprehensive visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Reading Ease by Category",
        "Grade Level Distribution",
        "Readability Heatmap",
        "Grade Level vs Reading Ease"
    ),
    specs=[
        [{"type": "bar"}, {"type": "box"}],
        [{"type": "heatmap"}, {"type": "scatter"}]
    ]
)

# Plot 1: Reading Ease by Category
fig.add_trace(
    go.Bar(
        x=df_texts['category'],
        y=df_texts['reading_ease'],
        marker_color=['#e74c3c', '#f39c12', '#3498db', '#9b59b6', '#2ecc71', '#1abc9c'],
        text=df_texts['reading_ease'].round(1),
        textposition='outside'
    ),
    row=1, col=1
)

# Plot 2: Grade Level Distribution (Box Plot)
all_grades = []
categories = []
for _, row in df_texts.iterrows():
    # Simulate multiple samples per category
    grades = [row['grade_level'] + np.random.normal(0, 0.5) for _ in range(10)]
    all_grades.extend(grades)
    categories.extend([row['category']] * 10)

fig.add_trace(
    go.Box(
        x=categories,
        y=all_grades,
        marker_color='#3498db'
    ),
    row=1, col=2
)

# Plot 3: Readability Heatmap
# Create matrix of different readability metrics
readability_matrix = []
metric_names = ['Flesch-Kincaid', 'Gunning Fog', 'SMOG', 'Coleman-Liau']

for _, row in df_texts.iterrows():
    text_metrics = [
        row['grade_level'],
        textstat.gunning_fog(row['text']),
        textstat.smog_index(row['text']) if len(sent_tokenize(row['text'])) >= 30 else 0,
        textstat.coleman_liau_index(row['text'])
    ]
    readability_matrix.append(text_metrics)

fig.add_trace(
    go.Heatmap(
        z=np.array(readability_matrix).T,
        x=df_texts['category'],
        y=metric_names,
        colorscale='RdYlGn_r',
        text=np.round(np.array(readability_matrix).T, 1),
        texttemplate="%{text}",
        textfont={"size": 10}
    ),
    row=2, col=1
)

# Plot 4: Grade Level vs Reading Ease Scatter
fig.add_trace(
    go.Scatter(
        x=df_texts['grade_level'],
        y=df_texts['reading_ease'],
        mode='markers+text',
        marker=dict(
            size=df_texts['word_count']/5,
            color=df_texts['grade_level'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(x=1.15)
        ),
        text=df_texts['category'],
        textposition="top center"
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    title_text="Comprehensive Readability Analysis",
    showlegend=False,
    height=800,
    width=1200
)

fig.update_xaxes(title_text="Category", row=1, col=1)
fig.update_yaxes(title_text="Reading Ease Score", row=1, col=1)
fig.update_yaxes(title_text="Grade Level", row=1, col=2)
fig.update_xaxes(title_text="Category", row=2, col=1)
fig.update_xaxes(title_text="Grade Level", row=2, col=2)
fig.update_yaxes(title_text="Reading Ease", row=2, col=2)

fig.show()

## 7. Text Simplification Analysis

In [ ]:
def simplify_text_analysis(original_text: str, simplified_versions: List[str]):
    """
    Analyze the effectiveness of text simplification.
    """
    results = []

    # Analyze original
    orig_fre = fk_metrics.flesch_reading_ease(original_text)
    orig_fkg = fk_metrics.flesch_kincaid_grade(original_text)

    results.append({
        'version': 'Original',
        'text': original_text[:100] + '...',
        'reading_ease': orig_fre,
        'grade_level': orig_fkg,
        'improvement': 0
    })

    # Analyze simplified versions
    for i, simplified in enumerate(simplified_versions, 1):
        simp_fre = fk_metrics.flesch_reading_ease(simplified)
        simp_fkg = fk_metrics.flesch_kincaid_grade(simplified)

        results.append({
            'version': f'Simplified {i}',
            'text': simplified[:100] + '...',
            'reading_ease': simp_fre,
            'grade_level': simp_fkg,
            'improvement': simp_fre - orig_fre
        })

    return pd.DataFrame(results)

# Example: Simplifying a complex academic text
complex_text = """The implementation of sophisticated machine learning algorithms has
revolutionized the methodological approaches utilized in contemporary computational
linguistics. Through the systematic application of transformer-based architectures,
researchers have demonstrated unprecedented capabilities in natural language
understanding and generation."""

simplified_versions = [
    """Machine learning has changed how we study language with computers.
    New AI models called transformers help computers understand and create
    human language better than ever before.""",

    """Smart computer programs now help us work with language. These programs
    can read and write like humans. This is a big step forward.""",

    """Computers can now understand language much better. They use new tools
    to read and write text. This helps in many ways."""
]

simplification_df = simplify_text_analysis(complex_text, simplified_versions)

print("\n📝 Text Simplification Analysis:")
print("="*80)
print(simplification_df[['version', 'reading_ease', 'grade_level', 'improvement']].round(2).to_string(index=False))

# Visualize simplification effectiveness
fig_simp = go.Figure()

fig_simp.add_trace(go.Bar(
    name='Reading Ease',
    x=simplification_df['version'],
    y=simplification_df['reading_ease'],
    marker_color='lightblue',
    yaxis='y',
))

fig_simp.add_trace(go.Scatter(
    name='Grade Level',
    x=simplification_df['version'],
    y=simplification_df['grade_level'],
    mode='lines+markers',
    marker_color='red',
    yaxis='y2'
))

fig_simp.update_layout(
    title='Text Simplification Effectiveness',
    xaxis=dict(title='Version'),
    yaxis=dict(title='Reading Ease Score', side='left'),
    yaxis2=dict(title='Grade Level', overlaying='y', side='right'),
    hovermode='x unified',
    height=400
)

fig_simp.show()

## 8. Production-Ready Readability Pipeline

In [ ]:
class ReadabilityEvaluationPipeline:
    """
    Production-ready pipeline for text readability evaluation with MLflow.
    """

    def __init__(self, experiment_name="readability-evaluation", tracking_uri=None):
        self.experiment_name = experiment_name
        if tracking_uri:
            mlflow.set_tracking_uri(tracking_uri)
        mlflow.set_experiment(experiment_name)

        self.fk_metrics = FleschKincaidMetrics()
        self.metrics = {
            "reading_ease": self.fk_metrics.create_reading_ease_metric(),
            "grade_level": self.fk_metrics.create_grade_level_metric(),
            "comprehensive": self.fk_metrics.create_comprehensive_readability_metric()
        }

    def evaluate_content(self,
                        content_name: str,
                        texts: List[str],
                        target_audience: str = "general",
                        metadata: Dict = None) -> Dict:
        """
        Evaluate content readability with comprehensive metrics.
        """
        with mlflow.start_run(run_name=content_name):
            # Log metadata
            mlflow.log_param("content_name", content_name)
            mlflow.log_param("num_texts", len(texts))
            mlflow.log_param("target_audience", target_audience)

            if metadata:
                for key, value in metadata.items():
                    mlflow.log_param(key, value)

            # Calculate comprehensive metrics
            results = self._calculate_all_metrics(texts)

            # Log metrics
            for metric_name, metric_value in results["aggregate"].items():
                mlflow.log_metric(metric_name, metric_value)

            # Generate and log report
            report = self._generate_readability_report(texts, results, target_audience)
            mlflow.log_dict(report, "readability_report.json")

            # Log recommendations
            recommendations = self._generate_recommendations(results, target_audience)
            mlflow.log_text("\n".join(recommendations), "recommendations.txt")

            return results

    def _calculate_all_metrics(self, texts: List[str]) -> Dict:
        """
        Calculate all readability metrics.
        """
        results = {
            "text_level": [],
            "aggregate": {},
            "distribution": {}
        }

        for text in texts:
            text_metrics = {
                'flesch_reading_ease': self.fk_metrics.flesch_reading_ease(text),
                'flesch_kincaid_grade': self.fk_metrics.flesch_kincaid_grade(text),
                'gunning_fog': textstat.gunning_fog(text),
                'smog_index': textstat.smog_index(text) if len(sent_tokenize(text)) >= 30 else 0,
                'coleman_liau': textstat.coleman_liau_index(text),
                'ari': textstat.automated_readability_index(text),
                'dale_chall': textstat.dale_chall_readability_score(text),
                'sentence_count': len(sent_tokenize(text)),
                'word_count': len(word_tokenize(text)),
                'syllable_count': sum(self.fk_metrics.count_syllables(w)
                                     for w in word_tokenize(text) if w.isalnum())
            }
            results["text_level"].append(text_metrics)

        # Calculate aggregates
        for metric_name in results["text_level"][0].keys():
            values = [t[metric_name] for t in results["text_level"]]
            results["aggregate"][f"{metric_name}_mean"] = np.mean(values)
            results["aggregate"][f"{metric_name}_std"] = np.std(values)
            results["aggregate"][f"{metric_name}_min"] = np.min(values)
            results["aggregate"][f"{metric_name}_max"] = np.max(values)

        # Calculate distribution
        fre_scores = [t['flesch_reading_ease'] for t in results["text_level"]]
        results["distribution"]["very_easy"] = sum(1 for s in fre_scores if s >= 90)
        results["distribution"]["easy"] = sum(1 for s in fre_scores if 80 <= s < 90)
        results["distribution"]["fairly_easy"] = sum(1 for s in fre_scores if 70 <= s < 80)
        results["distribution"]["standard"] = sum(1 for s in fre_scores if 60 <= s < 70)
        results["distribution"]["fairly_difficult"] = sum(1 for s in fre_scores if 50 <= s < 60)
        results["distribution"]["difficult"] = sum(1 for s in fre_scores if 30 <= s < 50)
        results["distribution"]["very_difficult"] = sum(1 for s in fre_scores if s < 30)

        return results

    def _generate_readability_report(self, texts: List[str], results: Dict,
                                    target_audience: str) -> Dict:
        """
        Generate comprehensive readability report.
        """
        report = {
            "summary": {
                "total_texts": len(texts),
                "target_audience": target_audience,
                "avg_reading_ease": results["aggregate"]["flesch_reading_ease_mean"],
                "avg_grade_level": results["aggregate"]["flesch_kincaid_grade_mean"],
                "readability_category": self._get_readability_category(
                    results["aggregate"]["flesch_reading_ease_mean"]
                )
            },
            "distribution": results["distribution"],
            "detailed_metrics": results["aggregate"],
            "sample_analysis": [
                {
                    "text_preview": text[:200] + "...",
                    "metrics": metrics
                }
                for text, metrics in zip(texts[:3], results["text_level"][:3])
            ]
        }

        return report

    def _get_readability_category(self, reading_ease: float) -> str:
        """
        Get readability category from Flesch Reading Ease score.
        """
        if reading_ease >= 90:
            return "Very Easy"
        elif reading_ease >= 80:
            return "Easy"
        elif reading_ease >= 70:
            return "Fairly Easy"
        elif reading_ease >= 60:
            return "Standard"
        elif reading_ease >= 50:
            return "Fairly Difficult"
        elif reading_ease >= 30:
            return "Difficult"
        else:
            return "Very Difficult"

    def _generate_recommendations(self, results: Dict, target_audience: str) -> List[str]:
        """
        Generate recommendations based on readability analysis.
        """
        recommendations = []

        avg_fre = results["aggregate"]["flesch_reading_ease_mean"]
        avg_grade = results["aggregate"]["flesch_kincaid_grade_mean"]

        # Target audience recommendations
        target_ranges = {
            "children": (90, 100, 1, 5),
            "young_adult": (70, 90, 6, 9),
            "general": (60, 80, 8, 12),
            "academic": (30, 60, 13, 18),
            "professional": (40, 70, 10, 16)
        }

        if target_audience in target_ranges:
            fre_min, fre_max, grade_min, grade_max = target_ranges[target_audience]

            if avg_fre < fre_min:
                recommendations.append(
                    f"Content is too difficult for {target_audience} audience. "
                    f"Consider simplifying sentences and using more common words."
                )
            elif avg_fre > fre_max:
                recommendations.append(
                    f"Content may be too simple for {target_audience} audience. "
                    f"Consider adding more sophisticated vocabulary and complex sentences."
                )

            if avg_grade < grade_min:
                recommendations.append(
                    f"Grade level ({avg_grade:.1f}) is below target range ({grade_min}-{grade_max})."
                )
            elif avg_grade > grade_max:
                recommendations.append(
                    f"Grade level ({avg_grade:.1f}) is above target range ({grade_min}-{grade_max})."
                )

        # General recommendations
        if results["aggregate"]["word_count_mean"] / results["aggregate"]["sentence_count_mean"] > 20:
            recommendations.append(
                "Average sentence length is high. Consider breaking long sentences into shorter ones."
            )

        if results["aggregate"]["syllable_count_mean"] / results["aggregate"]["word_count_mean"] > 2:
            recommendations.append(
                "Words have high syllable count. Consider using simpler vocabulary."
            )

        if not recommendations:
            recommendations.append(f"Content readability is appropriate for {target_audience} audience.")

        return recommendations

    def compare_versions(self, versions: Dict[str, List[str]]) -> pd.DataFrame:
        """
        Compare readability across multiple content versions.
        """
        comparison_results = []

        for version_name, texts in versions.items():
            results = self._calculate_all_metrics(texts)

            row = {
                "Version": version_name,
                "Reading Ease": results["aggregate"]["flesch_reading_ease_mean"],
                "Grade Level": results["aggregate"]["flesch_kincaid_grade_mean"],
                "Gunning Fog": results["aggregate"]["gunning_fog_mean"],
                "SMOG": results["aggregate"]["smog_index_mean"],
                "Category": self._get_readability_category(
                    results["aggregate"]["flesch_reading_ease_mean"]
                )
            }
            comparison_results.append(row)

        return pd.DataFrame(comparison_results)

# Demonstrate the pipeline
pipeline = ReadabilityEvaluationPipeline(experiment_name="readability-production-pipeline")

# Test with different content versions
content_versions = {
    "Original": [df_texts.iloc[3]['text']],  # Academic paper
    "Simplified_v1": [simplified_versions[0]],
    "Simplified_v2": [simplified_versions[1]],
    "Simplified_v3": [simplified_versions[2]]
}

# Compare versions
version_comparison = pipeline.compare_versions(content_versions)

print("\n🚀 Production Pipeline Results:")
print("="*90)
print(version_comparison.round(2).to_string(index=False))

# Identify best version for general audience
general_target = version_comparison[
    (version_comparison['Reading Ease'] >= 60) &
    (version_comparison['Reading Ease'] <= 80)
]

if not general_target.empty:
    best_version = general_target.iloc[0]
    print(f"\n🏆 Best Version for General Audience: {best_version['Version']}")
    print(f"   - Reading Ease: {best_version['Reading Ease']:.1f}")
    print(f"   - Grade Level: {best_version['Grade Level']:.1f}")
    print(f"   - Category: {best_version['Category']}")

## 9. A/B Testing for Content Readability

In [ ]:
def ab_test_readability(version_a: List[str], version_b: List[str],
                        confidence_level: float = 0.95) -> Dict:
    """
    Perform A/B testing on content readability.
    """
    # Calculate metrics for both versions
    fk = FleschKincaidMetrics()

    scores_a = [fk.flesch_reading_ease(text) for text in version_a]
    scores_b = [fk.flesch_reading_ease(text) for text in version_b]

    grades_a = [fk.flesch_kincaid_grade(text) for text in version_a]
    grades_b = [fk.flesch_kincaid_grade(text) for text in version_b]

    # Perform t-tests
    reading_ease_ttest = stats.ttest_ind(scores_a, scores_b)
    grade_level_ttest = stats.ttest_ind(grades_a, grades_b)

    # Calculate effect sizes (Cohen's d)
    def cohens_d(group1, group2):
        n1, n2 = len(group1), len(group2)
        var1, var2 = np.var(group1, ddof=1), np.var(group2, ddof=1)
        pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
        return (np.mean(group1) - np.mean(group2)) / pooled_std

    reading_ease_effect = cohens_d(scores_a, scores_b)
    grade_level_effect = cohens_d(grades_a, grades_b)

    results = {
        "version_a": {
            "reading_ease_mean": np.mean(scores_a),
            "reading_ease_std": np.std(scores_a),
            "grade_level_mean": np.mean(grades_a),
            "grade_level_std": np.std(grades_a)
        },
        "version_b": {
            "reading_ease_mean": np.mean(scores_b),
            "reading_ease_std": np.std(scores_b),
            "grade_level_mean": np.mean(grades_b),
            "grade_level_std": np.std(grades_b)
        },
        "statistical_tests": {
            "reading_ease_p_value": reading_ease_ttest.pvalue,
            "reading_ease_significant": reading_ease_ttest.pvalue < (1 - confidence_level),
            "reading_ease_effect_size": reading_ease_effect,
            "grade_level_p_value": grade_level_ttest.pvalue,
            "grade_level_significant": grade_level_ttest.pvalue < (1 - confidence_level),
            "grade_level_effect_size": grade_level_effect
        },
        "recommendation": ""
    }

    # Generate recommendation
    if results["statistical_tests"]["reading_ease_significant"]:
        if np.mean(scores_a) > np.mean(scores_b):
            results["recommendation"] = "Version A is significantly easier to read"
        else:
            results["recommendation"] = "Version B is significantly easier to read"
    else:
        results["recommendation"] = "No significant difference in readability"

    return results

# Create test versions
version_a = [
    "The research shows that exercise improves health. Regular activity helps prevent disease.",
    "Scientists found new ways to treat cancer. These methods work better than old ones.",
    "Climate change affects weather patterns. We need to reduce carbon emissions."
]

version_b = [
    "Contemporary investigations demonstrate that physical activity enhances physiological wellbeing.",
    "Researchers identified innovative therapeutic modalities for oncological interventions.",
    "Anthropogenic climate modifications influence meteorological phenomena globally."
]

# Run A/B test
ab_results = ab_test_readability(version_a, version_b)

print("\n🔬 A/B Testing Results:")
print("="*70)
print(f"\nVersion A:")
print(f"  Reading Ease: {ab_results['version_a']['reading_ease_mean']:.1f} ± {ab_results['version_a']['reading_ease_std']:.1f}")
print(f"  Grade Level: {ab_results['version_a']['grade_level_mean']:.1f} ± {ab_results['version_a']['grade_level_std']:.1f}")

print(f"\nVersion B:")
print(f"  Reading Ease: {ab_results['version_b']['reading_ease_mean']:.1f} ± {ab_results['version_b']['reading_ease_std']:.1f}")
print(f"  Grade Level: {ab_results['version_b']['grade_level_mean']:.1f} ± {ab_results['version_b']['grade_level_std']:.1f}")

print(f"\nStatistical Analysis:")
print(f"  Reading Ease p-value: {ab_results['statistical_tests']['reading_ease_p_value']:.4f}")
print(f"  Reading Ease Effect Size: {ab_results['statistical_tests']['reading_ease_effect_size']:.2f}")
print(f"  Grade Level p-value: {ab_results['statistical_tests']['grade_level_p_value']:.4f}")
print(f"  Grade Level Effect Size: {ab_results['statistical_tests']['grade_level_effect_size']:.2f}")

print(f"\n📊 Recommendation: {ab_results['recommendation']}")

## 10. Best Practices and Guidelines

### Key Takeaways:

1. **Understanding Flesch-Kincaid Metrics**:
   - **Reading Ease**: 0-100 scale (higher = easier)
   - **Grade Level**: U.S. education grade equivalent
   - Both metrics consider sentence length and syllable count
   - Complementary metrics provide fuller picture

2. **Target Audience Guidelines**:
   - **General Public**: 60-70 Reading Ease, Grade 8-9
   - **Children**: 90+ Reading Ease, Grade 1-5
   - **Young Adults**: 70-80 Reading Ease, Grade 6-8
   - **College**: 30-50 Reading Ease, Grade 13+
   - **Academic**: 0-30 Reading Ease, Grade 16+

3. **Content Optimization Strategies**:
   - **Simplify**: Shorter sentences, common words
   - **Structure**: Clear paragraphs, logical flow
   - **Vocabulary**: Match audience expectations
   - **Testing**: A/B test different versions

4. **MLflow Integration Benefits**:
   - Track readability changes over time
   - Compare content versions systematically
   - Automate readability monitoring
   - Generate actionable recommendations

5. **Production Deployment**:
   - Set readability thresholds for content
   - Automate content review workflows
   - Monitor readability drift
   - Generate alerts for out-of-range content

### Common Pitfalls to Avoid:
- Over-simplification losing meaning
- Ignoring audience expertise level
- Focusing on single metric only
- Not considering content purpose
- Neglecting cultural/regional differences

In [ ]:
# Final summary and MLflow UI instructions
print("\n📈 MLflow Tracking Summary:")
print("="*60)
print("To view all experiments and metrics in MLflow UI:")
print("\n1. Run in terminal:")
print("   mlflow ui --port 5000")
print("\n2. Open browser:")
print("   http://localhost:5000")
print("\n3. Navigate to experiments:")
print("   - flesch-kincaid-metrics-demo")
print("   - readability-production-pipeline")
print("\n✅ Demo completed successfully!")
print("\n🔗 Additional Resources:")
print("   - Flesch Reading Ease: https://en.wikipedia.org/wiki/Flesch_reading_ease")
print("   - Textstat Library: https://github.com/shivam5992/textstat")
print("   - Plain Language Guidelines: https://www.plainlanguage.gov/")